In [ ]:
# Import system libraries
import os
import sys

# Import data manipulation libraries
import pandas as pd
from shapely.geometry import Polygon

# Import geospatial libraries
import geopandas as gpd

# Import API library
import cdsapi # cds api client can be pip installed

# Import grib reader
import xarray as xr
import cfgrib

# Data request from API 
Use the cdsapi client to retrieve data in GRIB format from CEMS. 

Note, requires API key in config folder in the file $HOME/.cdsapirc. See https://cds.climate.copernicus.eu/api-how-to. Mine is included in the conda environment, and details below [REMINDER TO REMOVE FROM FINAL VERSION].

url: https://cds.climate.copernicus.eu/api/v2
key: 271227:e2fd9176-ac57-44fc-848f-5e3b06f8f09f

Note also that there is a limit on how many values you can download at the same time. I therefore split the downloads on the lead time variable.

In [ ]:
c = cdsapi.Client()
c.retrieve(
    'cems-fire-seasonal',
    {
        'format': 'grib',
        'variable': 'fire_weather_index',
        'release_version': '5',
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021', '2022',
            '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': '01',
        'leadtime_hour': '12',
        'area': [
            89.95, -179.95, -89.95,
            179.95,
        ],
    },
    'download12.grib')

In [ ]:
# Following code should repeat the above for other values of leadtime_hour
# They are equal to 12 + 24*n, so 12, 36, 60...

# List of leadtime_hour values
max_value = 5148

# Generate the sequence
leadtime_hours = [str(-12 + 24 * n) for n in range(1, (max_value + 12) // 24 + 1)]

for leadtime_hour in leadtime_hours:
    output_label = f'download{leadtime_hour}.grib'

c.retrieve(
    'cems-fire-seasonal',
    {
        'format': 'grib',
        'variable': 'fire_weather_index',
        'release_version': '5',
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021', '2022',
            '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': '01',
        'leadtime_hour': '36',
        'area': [
            89.95, -179.95, -89.95,
            179.95,
        ],
    },
    output_label)

In [ ]:
grib_data = cfgrib.open_datasets('assets\data\CEMS_indicators\download.grib')
grbs = xr.open_dataset(grib_file)

# Send to dataframe
grbs = grbs.to_dataframe()

# Merge the downloaded GRIB files
